<a href="https://colab.research.google.com/github/szhou52/HS-651/blob/main/Copy_of_30_day_readmission_Bert_sentence_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.1.3 spark-nlp==3.4.2

     |████████████████████████████████| 214.0 MB 8.5 kB/s 
     |████████████████████████████████| 142 kB 80.4 MB/s 
     |████████████████████████████████| 198 kB 75.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463484 sha256=e5c39340c86064f635e5f85e7af56c032791e37c4caf55370ff04ec0f805a7e2
  Stored in directory: /root/.cache/pip/wheels/ad/8e/49/44c110bb8e008d0778c6577d600d46047c6478ecca3f8f1517
Successfully built pyspark


In [2]:
import sparknlp

spark = sparknlp.start(gpu=True)

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  3.4.2
Apache Spark version:  3.1.3


In [3]:
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [4]:
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import StringIndexer

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
readmission_dir=pd.read_csv('gdrive/MyDrive/Colab_notebook/df_adm_dis_sum_not_cleaned.csv')

In [ ]:
readmission_dir['TEXT_AGG'][0]

"Admission Date:  [**2101-10-20**]     Discharge Date:  [**2101-10-31**]\n\nDate of Birth:   [**2025-4-11**]     Sex:  M\n\nService:  Medicine\n\nCHIEF COMPLAINT:  Admitted from rehabilitation for\nhypotension (systolic blood pressure to the 70s) and\ndecreased urine output.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old\nmale who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**]\nafter undergoing a left femoral-AT bypass graft and was\nsubsequently discharged to a rehabilitation facility.\n\nOn [**2101-10-20**], he presented again to the [**Hospital1 346**] after being found to have a systolic\nblood pressure in the 70s and no urine output for 17 hours.\nA Foley catheter placed at the rehabilitation facility\nyielded 100 cc of murky/brown urine.  There may also have\nbeen purulent discharge at the penile meatus at this time.\n\nOn presentation to the Emergency Department, the patient was\nwithout subjective complaints.  I

In [8]:
# Define text_clean function
import string
punc_numb=string.punctuation+'0123456789'
punc_numb=punc_numb.replace('!','')
punc_numb=punc_numb.replace('.','')
punc_numb=punc_numb.replace('?','')

In [ ]:
punc_numb

'"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~0123456789'

In [9]:
# Replace all the non-useful punctuations and numbers to space and only keep .!? as sentence separaters
def clean_text(text):
    text=text.replace("\n","")
    text=text.replace("\r","")

    t= text.maketrans(dict.fromkeys(punc_numb,""))
    text=text.translate(t)
    
    return(text)

In [10]:
from tqdm import tqdm
for i in tqdm(range(len(readmission_dir))):
    readmission_dir['TEXT_AGG'][i]=clean_text(readmission_dir['TEXT_AGG'][i])

  0%|          | 0/46308 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 46308/46308 [01:27<00:00, 531.65it/s]


In [27]:
from sklearn.model_selection import train_test_split
df_train, df_test=train_test_split(readmission_dir,test_size=0.2, random_state=42)

# sub-sampling the negatives (non-readmitted) on the training set
df_train_readm=df_train[df_train.READMISSION_STATUS=='Readmitted']
df_train_non_readm=df_train[df_train.READMISSION_STATUS=='Non-readmitted']
df_train_sub = pd.concat([df_train_readm, df_train_non_readm.sample(n = len(df_train_readm), random_state = 42)],axis = 0)

In [28]:
# Convert the pandas df to a spark df
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
train = spark.createDataFrame(df_train_sub)
test= spark.createDataFrame(df_test)

### Build up Pipelines

In [25]:
%%time
# Produce pipeline for data cleaning and sentence(discharge summary) embedding
document_assembler = DocumentAssembler() \
      .setInputCol("TEXT_AGG") \
      .setOutputCol("document")
    
sentence = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

Bert_sentence_embedding = BertSentenceEmbeddings.pretrained() \
  .setInputCols(["document","sentence"]) \
  .setOutputCol("sentence_bert_embeddings")

embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_bert_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx_label = StringIndexer(inputCol = "READMISSION_STATUS", outputCol = "label")

nlp_pipeline_Bert = Pipeline(
stages=[document_assembler, 
        sentence,
        Bert_sentence_embedding,
        embeddings_finisher,
        explodeVectors,
        label_stringIdx_label])

sent_small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]
CPU times: user 47.5 ms, sys: 2.06 ms, total: 49.6 ms
Wall time: 4.68 s


In [29]:
nlp_Bert_model= nlp_pipeline_Bert.fit(train)

In [30]:
nlp_Bert_model.write().overwrite().save('gdrive/MyDrive/Colab_Notebooks/Models_Pipelines/bert_sentence_readmission')

In [31]:
from pyspark.ml.pipeline import PipelineModel
nlp_Bert_model= PipelineModel.load("gdrive/MyDrive/Colab_Notebooks/Models_Pipelines/bert_sentence_readmission/")

In [32]:
# Transform training set
processed_bert=nlp_Bert_model.transform(train)

In [33]:
processed_bert.show()

+--------------------+----------+------+----------+-------+-------------------+-------------------+---------+--------------+--------------------+--------------------+---------+--------+--------+--------------+---------+-------------------+-------------------+----------+--------------------+--------------------+------+-------------------+--------------+-------------------+----------------+--------------------+------------------+--------------------+--------------------+--------------------+------------------------+----------------------------+-----+
|            features|Unnamed: 0|ROW_ID|SUBJECT_ID|HADM_ID|          ADMITTIME|          DISCHTIME|DEATHTIME|ADMISSION_TYPE|  ADMISSION_LOCATION|  DISCHARGE_LOCATION|INSURANCE|LANGUAGE|RELIGION|MARITAL_STATUS|ETHNICITY|          EDREGTIME|          EDOUTTIME| DIAGNOSIS|HOSPITAL_EXPIRE_FLAG|HAS_CHARTEVENTS_DATA|GENDER|                DOB|AGE_UPON_ADMIT|   READMISSION_TIME|READMISSION_TYPE|READMISSION_INTERVAL|READMISSION_STATUS|            TE

In [34]:
# Transform test set
processed_bert_test=nlp_Bert_model.transform(test)

In [36]:
from pyspark.sql.functions import collect_list
from pyspark.sql import functions as F
processed_bert_combined = processed_bert.groupby('HADM_ID').agg(collect_list('features').alias("features"),F.min(processed_bert.label))

In [40]:
processed_bert_test_combined = processed_bert_test.groupby('HADM_ID').agg(collect_list('features').alias("features"),F.min(processed_bert_test.label))

In [38]:
processed_bert_combined.show()

+-------+--------------------+----------+
|HADM_ID|            features|min(label)|
+-------+--------------------+----------+
| 102092|[[0.1668415367603...|       0.0|
| 116612|[[-0.054196644574...|       0.0|
| 123425|[[0.0912164077162...|       0.0|
| 125381|[[0.0198073573410...|       0.0|
| 147907|[[-0.038919840008...|       0.0|
| 194117|[[0.0261607021093...|       0.0|
| 199677|[[0.1052988022565...|       0.0|
| 139071|[[0.1651616245508...|       0.0|
| 166578|[[-0.045749936252...|       0.0|
| 169975|[[0.0914589464664...|       0.0|
| 191307|[[0.0486060269176...|       0.0|
| 191706|[[7.7287375461310...|       0.0|
| 194873|[[0.1190421208739...|       0.0|
| 101067|[[0.0421814545989...|       1.0|
| 101333|[[-0.046536531299...|       1.0|
| 111609|[[0.0084219211712...|       1.0|
| 116139|[[0.0648432374000...|       1.0|
| 136683|[[0.0154746845364...|       1.0|
| 139892|[[0.0389691367745...|       1.0|
| 167617|[[0.0627642571926...|       1.0|
+-------+--------------------+----

In [39]:
# Save the transformed training and test set as orc
processed_bert_combined.write.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_train")

In [41]:
processed_bert_test_combined.write.orc("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_test")

In [10]:
# Load in the saved transformed data
processed_bert_combined = spark.read.orc('gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_train/')
processed_bert_test_combined = spark.read.orc('gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_test/')

The dataframes are too big, so I decided to randomly split them and process them seperately.

In [31]:
bert_sen_tr1,bert_sen_tr2, bert_sen_tr3, bert_sen_tr4, bert_sen_tr5=processed_bert_combined.randomSplit([0.2, 0.2, 0.2, 0.2, 0.2])

In [56]:
# Define function to compute average embedding vector
def average_emb(df):
  for i in range(len(df)):
    new_embedding_list=[]
    embedding_list=df['features'][i]
    for k in range(len(embedding_list)):
      sentence_embedding=embedding_list[k][3]
      new_embedding_list.append(sentence_embedding)
    df['features'][i]=[sum(sub_list) / len(sub_list) for sub_list in zip(*new_embedding_list)]
  return(df)

In [32]:
pd_tr1=bert_sen_tr1.toPandas()
pd_tr1=average_emb(pd_tr1)

In [64]:
pd_tr2=bert_sen_tr2.toPandas()
pd_tr2=average_emb(pd_tr2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [65]:
pd_tr3=bert_sen_tr3.toPandas()
pd_tr3=average_emb(pd_tr3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [66]:
pd_tr4=bert_sen_tr4.toPandas()
pd_tr4=average_emb(pd_tr4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [67]:
pd_tr5=bert_sen_tr5.toPandas()
pd_tr5=average_emb(pd_tr5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [119]:
pd_train=pd.concat([pd_tr1,pd_tr2,pd_tr3,pd_tr4,pd_tr5]).reset_index(drop=True)

In [69]:
pd_train.to_csv("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_train.csv")

In [74]:
# Randomly sample 30% of the original test set. This is should be enough for the prediction evaluation
processed_bert_test_combined=processed_bert_test_combined.sample(withReplacement=False, fraction=0.3, seed=None)

In [76]:
bert_sen_te1,bert_sen_te2, bert_sen_te3, bert_sen_te4=processed_bert_test_combined.randomSplit([0.25, 0.25, 0.25, 0.25])

In [78]:
pd_te1=bert_sen_te1.toPandas()
pd_te1=average_emb(pd_te1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [79]:
pd_te2=bert_sen_te2.toPandas()
pd_te2=average_emb(pd_te2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [80]:
pd_te3=bert_sen_te3.toPandas()
pd_te3=average_emb(pd_te3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [81]:
pd_te4=bert_sen_te4.toPandas()
pd_te4=average_emb(pd_te4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [83]:
pd_test=pd.concat([pd_te1,pd_te2,pd_te3,pd_te4])

In [84]:
pd_test.to_csv("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_test.csv")

In [3]:
pd_train=pd.read_csv("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_train.csv")
pd_test=pd.read_csv("gdrive/MyDrive/Colab_notebook/transformed_data/bert_sentence_test.csv")

In [4]:
pd_train.label=pd_train['min(label)'].astype("int")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [5]:
pd_test.label=pd_test['min(label)'].astype("int")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [6]:
X_train=pd_train.features
y_train=pd_train['min(label)']

In [7]:
X_test=pd_test.features
y_test=pd_test['min(label)']

In [23]:
X_train_trans=[]
for doc in X_train:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)

In [24]:
X_test_trans=[]
for doc in X_test:
    embedding=doc[1:-1]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_test_trans.append(str_to_num_list)

## Modeling

In [25]:
model_bert_sentence=[]
Roc_auc_cv=[]
Roc_auc_test=[]

### Logistic Regression

In [29]:
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 4.0 MB/s 


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [31]:
# bert vs Logistic regression
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

param= dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

logistic_clf_bert = BayesSearchCV(estimator=LogisticRegression(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [32]:
logistic_clf_bert.fit(X_train_trans,y_train)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packa

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=LogisticRegression(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'C': [100, 10, 1.0, 0.1, 0.01], 'penalty': ['l2'],
                             'solver': ['newton-cg', 'lbfgs', 'liblinear']})

In [35]:
import pickle
with open('gdrive/MyDrive/Colab_notebook/Models_Pipelines/logistic_clf_bert_sent.pkl','wb') as f:
    pickle.dump(logistic_clf_bert,f)

In [36]:
with open('gdrive/MyDrive/Colab_notebook/Models_Pipelines/logistic_clf_bert_sent.pkl', 'rb') as f:
    logistic_clf_bert = pickle.load(f)

In [37]:
logistic_clf_bert_best=logistic_clf_bert.best_score_

In [38]:
logistic_clf_bert_best

0.6684509135299453

In [39]:
y_prob_logistic_clf_bert = logistic_clf_bert.predict_proba(X_test_trans)
roc_auc_y_prob_logistic_clf_bert=roc_auc_score(y_test, y_prob_logistic_clf_bert[:,1])

In [40]:
roc_auc_y_prob_logistic_clf_bert

0.6808877195268972

In [ ]:
model_bert.append("Logistic Regression")
Roc_auc_cv.append(logistic_clf_bert_best)
Roc_auc_test.append(roc_auc_y_prob_logistic_clf_bert)

### Naive Bayes

Naive Bayes doesn't allow negative inputs (there're negative numbers in word embeddings) so I'll skip the combination of GloVe and Naive Bayes

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_depth=[2, 3, 5, 10, 20]
min_samples_leaf=[5, 10, 20, 50, 100]
criterion=["gini", "entropy"]

param= dict(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

dec_tree_clf_bert = BayesSearchCV(estimator=DecisionTreeClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [ ]:
dec_tree_clf_bert.fit(X_train_trans,y_train)

In [ ]:
import pickle
with open('/Models_Pipelines/dec_tree_clf_bert.pkl','wb') as f:
    pickle.dump(dec_tree_clf_bert,f)

In [ ]:
with open('/Models_Pipelines/dec_tree_clf_bert.pkl', 'rb') as f:
    dec_tree_clf_bert = pickle.load(f)

In [ ]:
dec_tree_clf_bert_best=dec_tree_clf_bert.best_score_

In [ ]:
dec_tree_clf_bert_best

In [ ]:
y_prob_dec_tree_clf_bert = dec_tree_clf_bert.predict_proba(X_test_trans)
roc_auc_dec_tree_clf_bert=roc_auc_score(y_test, y_prob_dec_tree_clf_bert[:,1])

In [ ]:
roc_auc_dec_tree_clf_bert

In [ ]:
model_bert.append("Decision Tree")
Roc_auc_cv.append(dec_tree_clf_bert_best)
Roc_auc_test.append(roc_auc_dec_tree_clf_bert)

### Linear SVM

In [41]:
c_values=[100, 10, 1.0, 0.1, 0.01]

param= dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

lsvc_clf_bert = BayesSearchCV(estimator=LinearSVC(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [42]:
lsvc_clf_bert.fit(X_train_trans,y_train)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packa

BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=LinearSVC(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'C': [100, 10, 1.0, 0.1, 0.01]})

In [43]:
with open('gdrive/MyDrive/Colab_notebook/Models_Pipelines/lsvc_clf_bert_sent.pkl','wb') as f:
    pickle.dump(lsvc_clf_bert,f)

In [ ]:
with open('gdrive/MyDrive/Colab_notebook/Models_Pipelines/lsvc_clf_bert_sent.pkl', 'rb') as f:
    lsvc_clf_bert = pickle.load(f)

In [44]:
lsvc_clf_bert_best=lsvc_clf_bert.best_score_

In [45]:
lsvc_clf_bert_best

0.6676629449334933

In [46]:
y_dec_func_lsvc_clf_bert=lsvc_clf_bert.decision_function(X_test_trans)
roc_auc_lsvc_clf_bert=roc_auc_score(y_test, y_dec_func_lsvc_clf_bert)

In [47]:
roc_auc_lsvc_clf_bert

0.6793033788407024

In [ ]:
model_bert.append("Linear SVM")
Roc_auc_cv.append(lsvc_clf_bert_best)
Roc_auc_test.append(roc_auc_lsvc_clf_bert)

### KNN

In [ ]:
k_range = list(range(1, 31))
param= dict(n_neighbors=k_range)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

knn_clf_bert = BayesSearchCV(estimator=KNeighborsClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [ ]:
knn_clf_bert.fit(X_train_trans,y_train)

In [ ]:
with open('/Models_Pipelines/knn_clf_bert.pkl','wb') as f:
    pickle.dump(knn_clf_bert,f)

In [ ]:
with open('/Models_Pipelines/knn_clf_bert.pkl', 'rb') as f:
    knn_clf_bert = pickle.load(f)

In [ ]:
knn_clf_bert_best=knn_clf_bert.best_score_

In [ ]:
knn_clf_bert_best

In [ ]:
y_prob_knn_clf_bert = knn_clf_bert.predict_proba(X_test_trans)
roc_auc_knn_clf_bert=roc_auc_score(y_test,y_prob_knn_clf_bert[:,1])

In [ ]:
roc_auc_knn_clf_bert

In [ ]:
model_bert.append("KNN")
Roc_auc_cv.append(knn_clf_bert_best)
Roc_auc_test.append(roc_auc_knn_clf_bert)

### Random Forest

In [48]:
from sklearn.ensemble import RandomForestClassifier
max_depth=[5, 10, 15, 20]
min_samples_leaf=[5, 10, 20, 50, 100]
criterion=["gini", "entropy"]
n_estimators=[10,50,100,150]

param= dict(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion,n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

rand_for_clf_bert = BayesSearchCV(estimator=RandomForestClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [49]:
rand_for_clf_bert.fit(X_train_trans,y_train)

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=RandomForestClassifier(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': [5, 10, 15, 20],
                             'min_samples_leaf': [5, 10, 20, 50, 100],
                             'n_estimators': [10, 50, 100, 150]})

In [54]:
with open('gdrive/MyDrive/Colab_notebook/Models_Pipelines/rand_for_clf_bert_sent.pkl','wb') as f:
    pickle.dump(rand_for_clf_bert,f)

In [ ]:
with open('/Models_Pipelines/rand_for_clf_bert_sent.pkl', 'rb') as f:
    rand_for_clf_bert = pickle.load(f)

In [50]:
rand_for_clf_bert_best=rand_for_clf_bert.best_score_

In [51]:
rand_for_clf_bert_best

0.6687030446945098

In [52]:
y_prob_rand_for_clf_bert = rand_for_clf_bert.predict_proba(X_test_trans)
roc_auc_rand_for_clf_bert=roc_auc_score(y_test,y_prob_rand_for_clf_bert[:,1])

In [53]:
roc_auc_rand_for_clf_bert

0.6683840539571861

In [ ]:
model_bert.append("Random Forest")
Roc_auc_cv.append(rand_for_clf_bert_best)
Roc_auc_test.append(roc_auc_rand_for_clf_bert)

In [ ]:
result_bert=pd.DataFrame({'model_bert': model_bert, 'Roc_auc_cross_val': Roc_auc_cv,'Roc_auc_test':Roc_auc_test})
result_bert=result_bert.sort_values('Roc_auc_test')
result_bert.reset_index(drop = True)